# Ground Truth generation

# table of content
1) [Load stats](#load-stats)
2) [Show histograms and barplots](#show-histograms-and-barplots)
3) [Pie on heights](#pie-on-heights)

### Dependencies and general utils

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d
import laspy
import pdal
import json
from tqdm import tqdm

### Generation

In [40]:
# Loading sources
src_folder_instances = r"..\data\split_testing\gt\final_gt\color_grp_full_tile_633_final\instances"
src_original_prediction = r"..\data\split_testing\gt\predictions\color_grp_full_tile_633_out.laz"
src_folder_result = r"..\data\split_testing\gt\final_gt"

In [41]:
# Load original and reset/create gt columns
full_tile = laspy.read(src_original_prediction)
full_tile.remove_extra_dim('gt_semantic_segmentation')
full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_semantic_segmentation',type="uint16"))
full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_instance_segmentation',type="uint16"))
print(full_tile.x)

<ScaledArrayView([2572034.38 2572034.38 2572034.38 ... 2572134.26 2572134.26 2572134.26])>


In [42]:
# Loop on gt instances and set the correct values in the full tile
list_instances_src = [x for x in os.listdir(src_folder_instances) if x.endswith('.laz')]
rounding = 2
semantic_layer = np.zeros(len(full_tile))
instance_layer = np.zeros(len(full_tile))
for id_instance, instance_src in tqdm(enumerate(list_instances_src), total=len(list_instances_src)):
    instance = laspy.read(os.path.join(src_folder_instances, instance_src))
    coords = list(zip(np.round(instance.x, rounding), np.round(instance.y, rounding), np.round(instance.z, rounding)))
    mask = np.array([(x,y,z) in coords for x, y, z in zip(np.round(full_tile.x, rounding), np.round(full_tile.y, rounding), np.round(full_tile.z, rounding))])
    semantic_layer[mask] = 1
    instance_layer[mask] = id_instance + 1
    # print(np.sum(mask))
    # print(len(coords))
    # assert np.sum(mask) == len(coords)

setattr(full_tile, 'gt_semantic_segmentation', semantic_layer)
setattr(full_tile, 'gt_instance_segmentation', instance_layer)

# save file
new_file = os.path.join(os.path.join(src_folder_result), os.path.basename(src_original_prediction).split('.laz')[0] + '_gt.laz')
full_tile.write(new_file)

100%|██████████| 162/162 [09:33<00:00,  3.54s/it]


In [36]:
# verify results and save file
print("Semantic results:")
for cat in set(full_tile.gt_semantic_segmentation):
    print(f"\tVal {cat}: number of points = {len(full_tile.gt_semantic_segmentation[full_tile.gt_semantic_segmentation == cat])}")
print("Instances results:")
for cat in set(full_tile.gt_instance_segmentation):
    print(f"\tVal {cat}: number of points = {len(full_tile.gt_instance_segmentation[full_tile.gt_instance_segmentation == cat])}")

Semantic results:
	Val 0: number of points = 294154
	Val 1: number of points = 149430
Instances results:
	Val 0: number of points = 294154
	Val 1: number of points = 626
	Val 2: number of points = 228
	Val 3: number of points = 1084
	Val 4: number of points = 914
	Val 5: number of points = 1198
	Val 6: number of points = 1362
	Val 7: number of points = 1239
	Val 8: number of points = 96
	Val 9: number of points = 2312
	Val 10: number of points = 3209
	Val 11: number of points = 156
	Val 12: number of points = 491
	Val 13: number of points = 137
	Val 14: number of points = 3016
	Val 15: number of points = 547
	Val 16: number of points = 662
	Val 17: number of points = 464
	Val 18: number of points = 1494
	Val 19: number of points = 1319
	Val 20: number of points = 419
	Val 21: number of points = 1540
	Val 22: number of points = 186
	Val 23: number of points = 2231
	Val 24: number of points = 354
	Val 25: number of points = 744
	Val 26: number of points = 1907
	Val 27: number of points =

In [11]:

for col in list(full_tile.point_format.dimension_names):
    print(col)

X
Y
Z
intensity
return_number
number_of_returns
synthetic
key_point
withheld
overlap
scanner_channel
scan_direction_flag
edge_of_flight_line
classification
user_data
scan_angle
point_source_id
gps_time
red
green
blue
PredSemantic
gt_semantic_segmentation
PredInstance
gt_instance_segmentation


### Introducing clean samples into original

In [ ]:
src_origina